In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import nolds
import data
import mne

PROJ_ROOT = os.getenv('THESIS_ROOT')
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
REC_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'recplots'))
DIRECT_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'direct'))
GAF_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'gaf'))
print(PROJ_ROOT)
print(DATA_ROOT)
print(PROCESSED_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
# meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '50a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES).values

/home/kovar/thesis_project/
/home/kovar/thesis_project/data
/home/kovar/thesis_project/data/processed
Opening raw data file /home/kovar/thesis_project/data/processed/50a.fif...
This filename (/home/kovar/thesis_project/data/processed/50a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 16930 =      0.000 ...    67.720 secs
Ready.


<ipython-input-1-1e7cd7d477f5>:32: RuntimeWarning: This filename (/home/kovar/thesis_project/data/processed/50a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '50a.fif'))


# Models

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
from keras import initializers

from keras import backend as K
K.set_floatx('float32')
K.floatx()

'float32'

## Traditional CNN models

In [18]:
# Best model so far
def define_model_1(dropout_rate=dropout_rate):
    model = Sequential()
    model.add(Conv2D(8, (3, 3), input_shape=(image_height, image_width, num_channels), data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # model.add(Conv2D(4, (3, 3)))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

In [10]:
# VGG-like model
def define_model_2():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='Adam',
                  # optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [18]:
# Nikita's model
def define_model_3():
    model = Sequential()
    ki = initializers.RandomNormal(0, 0.1, 23)
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels),
                     kernel_initializer=ki))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.75))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.75))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer=ki))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    # model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(lr=1e-4, beta_1=0.999, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [19]:
# From 'Classification of Recurrence Plots’ Distance Matrices with a Convolutional Neural Network for
# Activity Recognition' paper
def define_model_4():
    model = Sequential()
    ki = initializers.RandomNormal()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels),
                     kernel_initializer=ki))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer=ki))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
# Temporary model with changes
def define_model_0(dropout_rate=dropout_rate):
    model = Sequential()
    model.add(Conv2D(8, (3, 3), input_shape=(image_height, image_width, num_channels), data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # model.add(Conv2D(4, (3, 3)))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    opt = optimizers.SGD(lr=0.01,  decay=1e-6, momentum=0.9, nesterov=True)
    opt = optimizers.RMSprop()
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

## Specialized models

In [39]:
from keras.layers import Conv1D, Reshape, Flatten
def define_model_fc_w_conv():
    inp = Input((imsize, imsize))
    
    layer = concatenate(
        [Dense(1, activation='relu', input_shape=(i+1,))(Lambda(lambda x: x[:,i,:i+1])(inp))
         for i in np.arange(imsize)], axis=-1)
    layer = Reshape((imsize, 1), input_shape=(imsize,))(layer)
    
    layer = Dropout(0.25)((Conv1D(19, 8, activation='relu')(layer)))
    
    layer = Flatten()(layer)
    
    layer = Dropout(0.25)((Dense(128, activation='relu')(layer)))
     
    outp = Dense(2, activation='sigmoid')(layer)
    
    model = Model(inputs=inp, outputs=outp)
    
    opt = optimizers.RMSprop()
    opt = optimizers.SGD(lr=0.01,  decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

define_model_fc_w_conv().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 256, 256)     0                                            
__________________________________________________________________________________________________
lambda_6401 (Lambda)            (None, 1)            0           input_26[0][0]                   
__________________________________________________________________________________________________
lambda_6402 (Lambda)            (None, 2)            0           input_26[0][0]                   
__________________________________________________________________________________________________
lambda_6403 (Lambda)            (None, 3)            0           input_26[0][0]                   
__________________________________________________________________________________________________
lambda_640

In [14]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Lambda, Dropout, concatenate, BatchNormalization
from keras import optimizers, regularizers
from keras.utils import plot_model

imsize = 256
batch_size = 32
def define_model_fc():
    inp = Input((imsize, imsize))
    
    layer = concatenate(
        [Dense(1, activation='relu', input_shape=(i+1,))(Lambda(lambda x: x[:,i,:i+1])(inp))
         for i in np.arange(imsize)], axis=-1)
    
    layer = Dropout(0.25)((Dense(128, activation='relu')(layer)))
    
    # layer = Dropout(0.25)((Dense(32, activation='relu')(layer)))
     
    outp = Dense(2, activation='sigmoid')(layer)
    
    model = Model(inputs=inp, outputs=outp)
    
    opt = optimizers.RMSprop()
    opt = optimizers.SGD(lr=0.01,  decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

define_model_fc().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256)     0                                            
__________________________________________________________________________________________________
lambda_1025 (Lambda)            (None, 1)            0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_1026 (Lambda)            (None, 2)            0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_1027 (Lambda)            (None, 3)            0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_102

In [41]:
import keras as k
import numpy as np
import tensorflow as tf

def square(x):
    import tensorflow as tf
    return tf.math.square(x)

def safe_log(x):
    import tensorflow as tf
    return tf.math.log(tf.clip_by_value(x, 1e-6, np.inf))

def define_bd_shal_model(
    in_chans = 19,
    input_time_length = 256,
    n_classes = 2,
    n_filters_time=40,
    n_filters_spat=40,
    filter_time_length=25,
    pool_time_length=75,
    pool_time_stride=15,
    final_conv_length=11,
    batch_norm=True,
    batch_norm_alpha=0.1,
    conv_nonlin=square,
    pool_mode='mean',
    pool_nonlin=safe_log,
    drop_prob=0.5,
):
    pool_class = dict(max=k.layers.MaxPooling2D, mean=k.layers.AveragePooling2D)[pool_mode]
    inp = k.layers.Input((in_chans, input_time_length, 1), name='input')
    
    layer = k.layers.Conv2D(
        filters=n_filters_time,
        kernel_size=(1, filter_time_length),
        use_bias=not batch_norm,
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        data_format='channels_last',
        input_shape=(imsize, imsize, 1),
        name='conv_time'
    )(inp)
    
    layer = k.layers.Conv2D(
        filters=n_filters_spat,
        kernel_size=(in_chans, 1),
        use_bias=not batch_norm,
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        name='conv_spat'
    )(layer)
    
    # Affine in orig - what it means? TODO: Apply here
    # Also, in orig some initialization
    if batch_norm:
        layer = k.layers.BatchNormalization(
            axis=-1,
            momentum=batch_norm_alpha,
            epsilon=1e-5,
            name='batch_norm',
            # There are other default params which may be diff. from orig.
        )(layer)
    
    layer = k.layers.Lambda(conv_nonlin, name='conv_nonlin')(layer)
    
    layer = pool_class(
        pool_size=(1, pool_time_length),
        strides=(1, pool_time_stride),
        name='pool'
    )(layer)
    
    layer = k.layers.Lambda(pool_nonlin, name='pool_nonlin')(layer)
    
    layer = k.layers.Dropout(drop_prob, name='drop')(layer)
    
    # TODO: Figure out final convolution length
    
    layer = k.layers.Conv2D(
        filters=n_classes,
        kernel_size=(1, final_conv_length),
        use_bias=True,
        activation='softmax',
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        name='conv_classifier'
    )(layer)
    
    layer = k.layers.Lambda(lambda x: x[:,0,0,:], name='squeeze')(layer)
    
    model = k.models.Model(inp, layer)
    model.compile(
        optimizer=k.optimizers.SGD(lr=0.01, momentum=0.99, decay=1e-5, nesterov=True),
        loss=k.losses.binary_crossentropy,
        metrics=['accuracy'],
        loss_weights=None,
        sample_weight_mode=None,
        weighted_metrics=None, 
        target_tensors=None
    )
    
    return model

define_bd_shal_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 19, 256, 1)        0         
_________________________________________________________________
conv_time (Conv2D)           (None, 19, 232, 40)       1000      
_________________________________________________________________
conv_spat (Conv2D)           (None, 1, 232, 40)        30400     
_________________________________________________________________
batch_norm (BatchNormalizati (None, 1, 232, 40)        160       
_________________________________________________________________
conv_nonlin (Lambda)         (None, 1, 232, 40)        0         
_________________________________________________________________
pool (AveragePooling2D)      (None, 1, 11, 40)         0         
_________________________________________________________________
pool_nonlin (Lambda)         (None, 1, 11, 40)         0         
__________

In [24]:
import keras as k
import numpy as np
import tensorflow as tf

def define_bd_deep_model(
        in_chans=19,
        n_classes=2,
        input_time_length=256,
        final_conv_length=1,
        n_filters_time=25,
        n_filters_spat=25,
        filter_time_length=10,
        pool_time_length=3,
        pool_time_stride=3,
        n_filters_2=50,
        filter_length_2=5,
        n_filters_3=100,
        filter_length_3=5,
        n_filters_4=200,
        filter_length_4=5,
        first_nonlin=k.activations.elu,
        first_pool_mode='max',
        first_pool_nonlin=(lambda x: x),
        later_nonlin=k.activations.elu,
        later_pool_mode='max',
        later_pool_nonlin=(lambda x: x),
        drop_prob=0.5,
        double_time_convs=False,
        split_first_layer=True,
        batch_norm=True,
        batch_norm_alpha=0.1,
        stride_before_pool=False,
):
    if stride_before_pool:
        conv_stride = pool_time_stride
        pool_stride = 1
    else:
        conv_stride = 1
        pool_stride = pool_time_stride
    # TODO: Behavior of their Avg differs
    pool_class_dict = dict(max=k.layers.MaxPool2D, mean=k.layers.AvgPool2D) # AvgPool2dWithConv)
    first_pool_class = pool_class_dict[first_pool_mode]
    later_pool_class = pool_class_dict[later_pool_mode]

    inp = k.layers.Input((in_chans, input_time_length, 1), name='input')
    
    layer = k.layers.Conv2D(
        filters=n_filters_time,
        kernel_size=(1, filter_time_length),
        use_bias=not batch_norm,
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        data_format='channels_last',
        input_shape=(imsize, imsize, 1),
        name='conv_time'
    )(inp)
    
    layer = k.layers.Conv2D(
        filters=n_filters_spat,
        kernel_size=(in_chans, 1),
        use_bias=not batch_norm,
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        name='conv_spat'
    )(layer)
    
    # Affine in orig - what it means? TODO: Apply here
    # Also, in orig some initialization
    if batch_norm:
        layer = k.layers.BatchNormalization(
            axis=-1,
            momentum=batch_norm_alpha,
            epsilon=1e-5,
            name='batch_norm',
            # There are other default params which may be diff. from orig.
        )(layer)
    
    layer = k.layers.Activation(first_nonlin, name='conv_nonlin')(layer)
    
    layer = first_pool_class(
        pool_size=(1, pool_time_length),
        strides=(1, pool_time_stride),
        name='pool'
    )(layer)
    
    layer = k.layers.Lambda(first_pool_nonlin, name='pool_nonlin')(layer)
    
    def add_conv_pool_block(layer, n_filters, filter_length, block_nr):
        suff = '_{:d}'.format(block_nr)
        layer = k.layers.Dropout(drop_prob, name='drop'+suff)(layer)

        layer = k.layers.Conv2D(
            filters=n_filters,
            kernel_size=(1, filter_length), # Not sure 
            strides=(1, conv_stride), # Not sure 
            use_bias=not batch_norm,
            kernel_initializer=k.initializers.glorot_uniform(),
            bias_initializer='zeros',
            name='conv'+suff
        )(layer)

        if batch_norm:
            layer = k.layers.BatchNormalization(
                axis=-1,
                momentum=batch_norm_alpha,
                epsilon=1e-5,
                name='batch_norm'+suff,
                # There are other default params which may be diff. from orig.
            )(layer)

        layer = k.layers.Activation(later_nonlin, name='nonlin'+suff)(layer)

        layer = later_pool_class(
            pool_size=(1, pool_time_length),
            strides=(1, pool_stride),
            name='pool'+suff
        )(layer)

        layer = k.layers.Lambda(later_pool_nonlin, name='pool_nonlin'+suff)(layer)

        return layer
    
    layer = add_conv_pool_block(layer, n_filters_2, filter_length_2, 2)
    layer = add_conv_pool_block(layer, n_filters_3, filter_length_3, 3)
    layer = add_conv_pool_block(layer, n_filters_4, filter_length_4, 4)
    
    layer = k.layers.Conv2D(
        filters=n_classes,
        kernel_size=(1, final_conv_length),
        use_bias=True,
        activation='softmax',
        kernel_initializer=k.initializers.glorot_uniform(),
        bias_initializer='zeros',
        name='conv_classifier'
    )(layer)
    
    layer = k.layers.Lambda(lambda x: x[:,0,0,:], name='squeeze')(layer)
    
    model = k.models.Model(inp, layer)
    model.compile(
        optimizer=k.optimizers.SGD(lr=0.01, momentum=0.99, decay=1e-5, nesterov=True),
        loss=k.losses.binary_crossentropy,
        metrics=['accuracy'],
        loss_weights=None,
        sample_weight_mode=None,
        weighted_metrics=None, 
        target_tensors=None
    )
    
    return model

model = define_bd_deep_model()

model.summary()

from keras.utils import plot_model
plot_model(model, to_file='/home/kovar/deep_model.png', show_layer_names=True, show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 19, 256, 1)        0         
_________________________________________________________________
conv_time (Conv2D)           (None, 19, 247, 25)       250       
_________________________________________________________________
conv_spat (Conv2D)           (None, 1, 247, 25)        11875     
_________________________________________________________________
batch_norm (BatchNormalizati (None, 1, 247, 25)        100       
_________________________________________________________________
conv_nonlin (Activation)     (None, 1, 247, 25)        0         
_________________________________________________________________
pool (MaxPooling2D)          (None, 1, 82, 25)         0         
_________________________________________________________________
pool_nonlin (Lambda)         (None, 1, 82, 25)         0         
__________

# Prepare data

In [26]:
def f(row, col):
    if row[col] <= 16:
        return -1
    if row[col] <= 28:
        return 0
    else:
        return 1
def ff(row, col, n=5):
    for i in range(n):
        if row[col] <= metapkl.quantile((i+1)/n)[col]:
            return i
    
metapklpath = os.path.join(LABELED_ROOT, 'processed', 'meta', 'meta.pkl')
metapkl = pd.read_pickle(metapklpath)
metapkl['dep'] = metapkl.apply(lambda row: f(row, 'sc'), axis=1)
metapkl['dep_bef'] = metapkl.apply(lambda row: f(row, 'sc_bef'), axis=1)
metapkl['dep_aft'] = metapkl.apply(lambda row: f(row, 'sc_aft'), axis=1)
metapkl = metapkl.astype({'dep': 'category', 'dep_bef': 'category', 'dep_aft': 'category'})                                           
print(metapkl['dep'].value_counts())
print(metapkl['dep_bef'].value_counts())
print(metapkl['dep_aft'].value_counts())

def f(row, col):
    if row[col] <= metapkl.loc[(slice(None), 'a'), :].quantile(0.42)['change']:
        return -1
    if row[col] <= metapkl.loc[(slice(None), 'a'), :].quantile(0.56)['change']:
        return 0
    else:
        return 1
    
metapkl['resp'] = metapkl.apply(lambda row: f(row, 'change'), axis=1)
metapkl = metapkl.astype({'resp': 'category'})                                           
print(metapkl.loc[(slice(None), 'a'), 'resp'].value_counts())
pd.to_pickle(metapkl, metapklpath)
metapkl

 0    148
-1     62
 1     56
Name: dep, dtype: int64
0    170
1     96
Name: dep_bef, dtype: int64
 0    126
-1    124
 1     16
Name: dep_aft, dtype: int64
 1    59
-1    57
 0    17
Name: resp, dtype: int64


resp b/a sex  age    sc  sc_bef  sc_aft dep dep_bef dep_aft  \
patient trial                                                               
1       a        1   0   0   49  26.0      26      13   0       0      -1   
        b        1   1   0   49  13.0      26      13  -1       0      -1   
2       a       -1   0   0   36  31.0      31      30   1       1       1   
        b       -1   1   0   36  30.0      31      30   1       1       1   
3       a        1   0   0   48  25.0      25       7   0       0      -1   
        b        1   1   0   48   7.0      25       7  -1       0      -1   
4       a       -1   0   0   55  26.0      26      25   0       0       0   
        b       -1   1   0   55  25.0      26      25   0       0       0   
5       a       -1   0   0   35  26.0      26      37   0       0       1   
        b       -1   1   0   35  37.0      26      37   1       0       1   
6       a        0   0   0   37  31.0      31      17   1       1       0   
        b        0   1   0   37  17.0      31      17   0       1       0   
7       a       -1   0   0   63  30.0      30      25   1       1       0   
        b       -1   1   0   63  25.0      30      25   0       1       0   
8       a        1   0   1   50  24.0      24      11   0       0      -1   
        b        1   1   1   50  11.0      24      11  -1       0      -1   
9       a       -1   0   1   59  34.0      34      27   1       1       0   
        b       -1   1   1   59  27.0      34      27   0       1       0   
10      a        1   0   0   49  36.0      36      17   1       1       0   
        b        1   1   0   49  17.0      36      17   0       1       0   
11      a       -1   0   1   53  27.0      27      26   0       0       0   
        b       -1   1   1   53  26.0      27      26   0       0       0   
12      a        1   0   1   59  30.0      30      12   1       1      -1   
        b        1   1   1   59  12.0      30      12  -1       1      -1   
13      a        1   0   0   57  26.0      26       7   0       0      -1   
        b        1   1   0   57   7.0      26       7  -1       0      -1   
14      a        0   0   0   30  28.0      28      17   0       0       0   
        b        0   1   0   30  17.0      28      17   0       0       0   
15      a        1   0   0   35  25.0      25       3   0       0      -1   
        b        1   1   0   35   3.0      25       3  -1       0      -1   
...            ...  ..  ..  ...   ...     ...     ...  ..     ...     ...   
119     a        1   0   1   61  23.0      23       4   0       0      -1   
        b        1   1   1   61   4.0      23       4  -1       0      -1   
120     a       -1   0   0   41  28.0      28      19   0       0       0   
        b       -1   1   0   41  19.0      28      19   0       0       0   
121     a       -1   0   0   34  24.0      24      26   0       0       0   
        b       -1   1   0   34  26.0      24      26   0       0       0   
122     a        0   0   1   55  35.0      35      19   1       1       0   
        b        0   1   1   55  19.0      35      19   0       1       0   
123     a       -1   0   1   54  24.0      24      25   0       0       0   
        b       -1   1   1   54  25.0      24      25   0       0       0   
124     a       -1   0   0   52  30.0      30      23   1       1       0   
        b       -1   1   0   52  23.0      30      23   0       1       0   
125     a        1   0   0   56  26.0      26      11   0       0      -1   
        b        1   1   0   56  11.0      26      11  -1       0      -1   
126     a       -1   0   0   40  25.0      25      17   0       0       0   
        b       -1   1   0   40  17.0      25      17   0       0       0   
127     a       -1   0   0   35  25.0      25      21   0       0       0   
        b       -1   1   0   35  21.0      25      21   0       0       0   
128     a        1   0   0   51  21.0      21       9   0       0      -1   
        b 

In [27]:
import numpy as np

def remove_middle(filenames, labels):
    ta = zip(filenames, labels)
    ta = np.array([(t, l) for t, l in ta], dtype=[('fname', 'S100'), ('label', 'int8')])
    ta = ta[:][(ta['label'] == -1) | (ta['label'] == 1)]
    ta['label'][ta['label'] == -1] = 0
    return ta['fname'].astype(str, copy=False), ta['label']

In [28]:
from sklearn.model_selection import GridSearchCV, train_test_split
from data.data_files import CHANNEL_NAMES, DataKind, files_builder
from keras.utils import to_categorical

fb = files_builder(DataKind('direct'), subfolder=())
seed = 123
fns = [fn[1] for fn in fb.file_names(include_path=True, subfolder=(), recursive=True)]
filenames, labels = remove_middle(fns, fb.get_labels(fns, label='resp'))
# labels = to_categorical(labels)
unique, counts = np.unique(labels, return_counts=True)
print('Overall samples: ', dict(zip(unique, counts)))
training_filenames, test_filenames, training_labels, test_labels = \
    train_test_split(filenames, labels, test_size=0.2, random_state=seed)
training_filenames, validation_filenames, training_labels, validation_labels = \
    train_test_split(training_filenames, training_labels, test_size=0.2, random_state=seed)
assert len(training_filenames) == len(training_labels)
assert len(validation_filenames) == len(validation_labels)
unique, counts = np.unique(training_labels, return_counts=True)
print('Training samples: ', dict(zip(unique, counts)))
unique, counts = np.unique(test_labels, return_counts=True)
print('Testing samples: ', dict(zip(unique, counts)))
unique, counts = np.unique(validation_labels, return_counts=True)
print('Validation samples: ', dict(zip(unique, counts)))
labels = to_categorical(labels, dtype=labels.dtype)
training_labels = to_categorical(training_labels, dtype=training_labels.dtype)
validation_labels = to_categorical(validation_labels, dtype=validation_labels.dtype)
test_labels = to_categorical(test_labels, dtype=test_labels.dtype)

Overall samples:  {0: 8366, 1: 8495}
Training samples:  {0: 5348, 1: 5442}
Testing samples:  {0: 1669, 1: 1704}
Validation samples:  {0: 1349, 1: 1349}


In [22]:
np.load(filenames[0]).shape

(19, 256)

# Train

In [29]:
from keras.utils import Sequence

ind = [False, False, False, False, False,
       False, False, False, False, False, 
       False, False, False, False, False, 
       True, False, False, False]

class batch_generator(Sequence):

    def __init__(self, filenames, labels, batch_size):
        self.filenames, self.labels = filenames, labels
        self.n = len(self.filenames)
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))
        # return len(self.filenames) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min(self.n, (idx + 1) * self.batch_size)]
        batch_y = self.labels[idx * self.batch_size:min(self.n, (idx + 1) * self.batch_size)]
        # assert len(batch_x) == batch_size, batch_x
        # assert len(batch_y) == batch_size, batch_y

        return np.array([
            np.load(file_name)[:,:, None] \
            # np.expand_dims(np.load(file_name), axis=-1) \
            # np.expand_dims(np.mean(np.load(file_name)[:, :, ind], axis=-1), axis=-1) \
            for file_name in batch_x]), batch_y

In [30]:
image_height, image_width = 256, 256
# image_height, image_width = 256, 19
num_channels = 1
batch_size = 128
num_epochs= 200
dropout_rate = 0.5

def plot_history(history):
    import matplotlib.pyplot as plt
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [20]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

training_batch_generator = batch_generator(training_filenames, training_labels, batch_size)
validation_batch_generator = batch_generator(validation_filenames, validation_labels, batch_size)

mchp_filepath = '/home/kovar/thesis_project/data/models/weights_deep_best.hdf5'
model = define_bd_shal_model(in_chans=19, input_time_length=256)
mcp = ModelCheckpoint(mchp_filepath, save_best_only=True, monitor='val_acc', save_weights_only=False)
history = model.fit_generator(generator=training_batch_generator,
                                      steps_per_epoch=len(training_batch_generator),
                                      epochs=num_epochs,
                                      verbose=1,
                                      validation_data=validation_batch_generator,
                                      validation_steps=len(validation_batch_generator),
                                      callbacks=[
                                          mcp,
                                      #     # EarlyStopping(patience=4),
                                      #     ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, 
                                      #                       min_lr=0.001)
                                      ],
                                      use_multiprocessing=True,
                                      workers=8,
                                      max_queue_size=32)
plot_history(history)

Epoch 1/200


ValueError: Error when checking input: expected input to have shape (19, 512, 1) but got array with shape (512, 19, 1)

In [13]:
from keras.models import load_model
model = load_model(mchp_filepath)
test_batch_generator = batch_generator(test_filenames, test_labels, batch_size)
model.evaluate_generator(test_batch_generator, steps=None, max_queue_size=10, workers=1, 
                         use_multiprocessing=False, verbose=0)

[0.39847876710705116, 0.81440853848147743]

## Evaluate with cross validation

In [31]:
# For being able to use multiprocessing with generators and callbacks, use export HDF5_USE_FILE_LOCKING=FALSE
import os
print(os.getenv('HDF5_USE_FILE_LOCKING'))

FALSE


In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy
from sklearn.model_selection import GridSearchCV, train_test_split
from data.data_files import CHANNEL_NAMES, DataKind, files_builder
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

seed = 123
numpy.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

cvscores = []
fb = files_builder(DataKind('direct'), subfolder=())
fns = [fn[1] for fn in fb.file_names(include_path=True, subfolder=(), recursive=True)]
filenames, labels = remove_middle(fns, fb.get_labels(fns, label='resp'))
for i, (train, test) in enumerate(kfold.split(filenames, labels)):
    cat_labels = to_categorical(labels)
    
    training_filenames, validation_filenames, training_labels, validation_labels = \
        train_test_split(filenames[train], cat_labels[train], test_size=0.2, random_state=seed)
    
    training_batch_generator = batch_generator(training_filenames, training_labels, batch_size)
    validation_batch_generator = batch_generator(validation_filenames, validation_labels, batch_size)
    test_batch_generator = batch_generator(filenames[test], cat_labels[test], batch_size)
    
    # mchp_filepath = '/home/kovar/thesis_project/data/models/weights_deep.{epoch:02d}-{val_loss:.2f}.hdf5'
    mchp_filepath = '/home/kovar/thesis_project/data/models/weights_shallow_best.hdf5'
    model = define_bd_deep_model(in_chans=19, input_time_length=256)
    mcp = ModelCheckpoint(mchp_filepath, save_best_only=True, monitor='val_acc', save_weights_only=False)
    print('FOLD ', i+1)
    history = model.fit_generator(generator=training_batch_generator,
                                          steps_per_epoch=len(training_batch_generator),
                                          epochs=num_epochs,
                                          verbose=0,
                                          validation_data=validation_batch_generator,
                                          validation_steps=len(validation_batch_generator),
                                          callbacks=[
                                              mcp,
                                          #     # EarlyStopping(patience=4),
                                          #     ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, 
                                          #                       min_lr=0.001)
                                          ],
                                          use_multiprocessing=True,
                                          workers=8,
                                          max_queue_size=32)
    
    model = load_model(mchp_filepath)
    scores = model.evaluate_generator(test_batch_generator, steps=None, max_queue_size=10, workers=1, 
                             use_multiprocessing=False, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

FOLD  1


## Other

In [35]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    print('Total paramaters: {:,}'.format(trainable_count + non_trainable_count))
    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes
get_model_memory_usage(128, define_model_1())

Total paramaters: 12,694.0


2.518